In [ ]:
# Importação de bibliotecas
import os
import numpy as np  # Álgebra linear
import pandas as pd  # Processamento de dados e leitura de arquivos CSV

# Exibição dos arquivos disponíveis no diretório de entrada
input_dir = '/kaggle/input'
for dirname, _, filenames in os.walk(input_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Observações:
# - Você pode gravar até 20GB no diretório atual (/kaggle/working/) e os arquivos serão preservados
#   ao criar uma versão usando "Save & Run All".
# - Arquivos temporários podem ser gravados em /kaggle/temp/, mas não serão salvos fora da sessão atual.

In [ ]:
# Importações de bibliotecas
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Exibe a versão do TensorFlow
print(tf.__version__)

# Configuração dos caminhos para os conjuntos de dados
train_dir = Path('../input/fruit-and-vegetable-image-recognition/train')
test_dir = Path('../input/fruit-and-vegetable-image-recognition/test')
val_dir = Path('../input/fruit-and-vegetable-image-recognition/validation')

# Listagem de arquivos
train_filepaths = list(train_dir.glob(r'**/*.jpg'))
test_filepaths = list(test_dir.glob(r'**/*.jpg'))
val_filepaths = list(val_dir.glob(r'**/*.jpg'))

# Função para processamento de imagens
def image_processing(filepath):
    """
    Cria um DataFrame contendo os caminhos dos arquivos e os rótulos das imagens.
    """
    labels = [str(filepath[i]).split("/")[-2] for i in range(len(filepath))]
    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatena caminhos e rótulos e embaralha o DataFrame
    df = pd.concat([filepath, labels], axis=1).sample(frac=1).reset_index(drop=True)
    return df

# Criação dos DataFrames de treino, teste e validação
train_df = image_processing(train_filepaths)
test_df = image_processing(test_filepaths)
val_df = image_processing(val_filepaths)

# Informações sobre o conjunto de treino
print('-- Conjunto de Treino --\n')
print(f'Número de imagens: {train_df.shape[0]}\n')
print(f'Número de rótulos únicos: {len(train_df.Label.unique())}\n')
print(f'Rótulos: {train_df.Label.unique()}')

# Exibe as primeiras 5 linhas do DataFrame de treino
print(train_df.head(5))

# Visualização de amostras do conjunto de dados
df_unique = train_df.drop_duplicates(subset=["Label"]).reset_index()
fig, axes = plt.subplots(nrows=6, ncols=6, figsize=(8, 7), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df_unique.Filepath[i]))
    ax.set_title(df_unique.Label[i], fontsize=12)

plt.tight_layout(pad=0.5)
plt.show()

# Geradores de imagens com pré-processamento
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

# Configuração dos geradores para treino, validação e teste
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_images = train_generator.flow_from_dataframe(
    dataframe=val_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

# Configuração do modelo pré-treinado MobileNetV2
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
pretrained_model.trainable = False

# Construção do modelo
inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(36, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Treinamento do modelo
history = model.fit(
    train_images,
    validation_data=val_images,
    batch_size=32,
    epochs=5,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=2,
            restore_best_weights=True
        )
    ]
)

# Predição nos dados de teste
pred = model.predict(test_images)
pred = np.argmax(pred, axis=1)

# Mapeamento dos rótulos
labels = {v: k for k, v in train_images.class_indices.items()}
pred_labels = [labels[k] for k in pred]

# Função para predição em uma imagem individual
def output(location):
    """
    Faz a predição para uma imagem fornecida.
    """
    img = load_img(location, target_size=(224, 224, 3))
    img = img_to_array(img) / 255
    img = np.expand_dims(img, axis=0)
    answer = model.predict(img)
    y_class = answer.argmax(axis=-1)
    res = labels[int(y_class[0])]
    return res

# Teste com uma imagem de exemplo
img = output('../input/fruit-and-vegetable-image-recognition/test/cabbage/Image_1.jpg')
print(img)

# Salvar o modelo treinado
model.save('model.h5')